# Итоговая аттестация
## Обучение предобученной YOLOv8 nano для задачи детекции на собственных данных, подготовленных ранее.

**В рамках выполнения задания используем датасет, полученный ранее. Для получения лучшего результата обучения для уменьшения вероятности переобучение на одном видео, добавим 19 изображений из другого видео, также разметив.**

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import torch
import cv2
import os
import shutil
import natsort as ns
import json
import albumentations as A
from ultralytics import YOLO
from ultralytics import settings
from ultralytics.utils import plotting
import random

## Выполнение работы

### 1. Добавление размеченных изображений в выборку

#### 1.1 Извлечение кадров видео

**Функция извлечения кадров из видео**

In [2]:
def video2frames(src, out, sample, xr, yr, count_frames):
    
    if not os.path.exists(out):
        os.mkdir(out)
    
    cap = cv2.VideoCapture(src)
    
    if not cap.isOpened(): 
        print('Ошибка чтения видео-файла')
    
    i, s = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret and s < count_frames:
            if i % sample == 0:
                frame = cv2.resize(frame, (xr, yr), cv2.INTER_NEAREST)
                cv2.imwrite(os.path.join(out, os.path.split(src)[-1].split('.')[0] + '_' + str(i) + '.jpg'), frame)
                s += 1
            i += 1
        else:
            break
    cap.release()
        
    return f'Задача завершена. Сохранено кадров: {s}.'

Функция читает видео по па адресу **src** по кадрам и сохраняет их как изображения в директорию **out**. Настраиваем выборку не всех, а каждого i-того кадра (параметр **sample** определяем ниже)

**Загрузка видео, определение директорий, параметров сохранения кадров**

In [3]:
src = 'for_add_img.mp4'
out = os.path.join(os.curdir, 'frames')
cap = cv2.VideoCapture(src)

new_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
new_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

count_frames = 50 # видео наполнено интересующими объектами первую минуту
sample = int(cap.get(cv2.CAP_PROP_FPS)) # вырезка кадров каждую секунду

**Запуск скрипта**

In [4]:
%%time

video2frames(src, out, sample, new_x, new_y, count_frames)

CPU times: total: 1min 35s
Wall time: 27.2 s


'Задача завершена. Сохранено кадров: 50.'

#### 1.2 Создание вырезок 640х640

**Из полученных кадров делаем вырезки 640х640 из собранных кадров, так как именно на 640х640 обучалась YOLO.**

Для получения вырезок используем функцию **click_sampler** и вспомогательную функцию **set_coord_center**

In [5]:
def set_coord_center(xc, yc, size, img_x, img_y):
    limit_x = size[0] // 2
    limit_y = size[1] // 2
    if xc < limit_x:
        xc = limit_x
    if xc > img_x - limit_x:
        xc = img_x - limit_x
    if yc < limit_y:
        yc = limit_y
    if yc > img_y - limit_y:
        yc = img_y - limit_y
    return xc, yc

In [6]:
def click_sampler(name, size, out):
    
    if not os.path.exists(out):
        os.mkdir(out)
        
    if size[0] <= 0 or size[1] <= 0:
        raise ValueError ('tile size shuld be >= 0')
  
    img = cv2.imread(name)
    
    if size[0] > img.shape[0] or size[1] > img.shape[1]:
        print('Warnng: tile size > img size')
        pass
    
    img_c = img.copy()

    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.resizeWindow(name, img.shape[1], img.shape[0])

    coords = []

    def mouse_click(event, x, y, flags, param):
        
        if event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(img, (x, y), 10, (0, 0, 255), -1)
            x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
            xw, yh = x + int(size[0]/2), y + int(size[1]/2)
            cv2.rectangle(img, (x0, y0), (xw, yh), (0, 0, 255), 3)
            cv2.imshow(name, img)
            coords.append((x, y))

        return coords

    cv2.setMouseCallback(name, mouse_click)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    t = 0
    nm, ext = os.path.basename(name)[:-4], os.path.basename(name)[-4:]
    
    for coord in coords:
        x, y = coord[0], coord[1]
        x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
        xw, yh = x + int(size[0]/2), y + int(size[1]/2)
        
        if x0 < 0:
            x0 = 0 
        if y0 < 0:
            y0 = 0
            
        tile = img_c[y0:yh, x0:xw]
        print(out + '/' + os.path.basename(nm) + '_' + str(x)+ 'x' + str(y) + ext)
        cv2.imwrite(out + '/' + os.path.basename(nm)[:-1] + '_' + str(x)+ 'x' + str(y) + '.' + ext, tile)
                    
        t += 1
        
    return f'{t} tiles were generated'

**Применяем скрипт click_sampler итеративно для всех изображений в папке frame**

In [7]:
pth = 'frames'
out = '640x640'

out_pth = os.path.join(pth, out)

if not os.path.exists(out_pth):
    os.mkdir(out_pth)
for file in ns.natsorted(os.listdir(pth)):
    f = os.path.join(pth, file)
    if os.path.isfile(f): # для исключения ошибки при попытке обработать директорию
        print(os.path.join(out_pth, os.path.basename(file)))
        click_sampler(f, (640,640), out_pth)

frames\640x640\for_add_img_0.jpg
frames\640x640\for_add_img_24.jpg
frames\640x640\for_add_img_48.jpg
frames\640x640\for_add_img_72.jpg
frames\640x640\for_add_img_96.jpg
frames\640x640\for_add_img_120.jpg
frames\640x640\for_add_img_144.jpg
frames\640x640\for_add_img_168.jpg
frames\640x640\for_add_img_192.jpg
frames\640x640\for_add_img_216.jpg
frames\640x640\for_add_img_240.jpg
frames\640x640\for_add_img_264.jpg
frames\640x640\for_add_img_288.jpg
frames\640x640\for_add_img_312.jpg
frames\640x640\for_add_img_336.jpg
frames\640x640\for_add_img_360.jpg
frames\640x640\for_add_img_384.jpg
frames\640x640\for_add_img_408.jpg
frames\640x640\for_add_img_432.jpg
frames\640x640\for_add_img_456.jpg
frames\640x640\for_add_img_480.jpg
frames\640x640\for_add_img_504.jpg
frames\640x640\for_add_img_528.jpg
frames\640x640\for_add_img_552.jpg
frames\640x640\for_add_img_576.jpg
frames\640x640\for_add_img_600.jpg
frames\640x640\for_add_img_624.jpg
frames\640x640\for_add_img_648.jpg
frames\640x640\for_add_img

#### 1.3 Разметка данных:

**Для разметки изображений используем VGG Image Annotation (VIA) - браузерный инструмент разметки изображений для создания обучающих выборок.
Инструмент обладает рядом преимуществ, однако для возможности загрузки размеченных изображений в YOLO, будет необходим скрипт *VIA2YOLO_detect*, приведенный ниже.**

**При разметке изображений соблюдены следующие принципы:**
* bounding box отрисовывались на каждо различимом объекте, независимо от его удаленности в кадре;
* во мзбежание дополнительного "шума" при обучении исключались из разметки объекты, не различимые глазу даже при увеличении, при этом были интуитивно понятно, что это автомобили (за исключением случаев, когда явно различались характерные признаки автомобиля, например - включенные фары, видимое разделение кузова и остекления, выступающие зеркала, четкая видимость колес и т.п.);
* на некоторых кадрах присутствует строительная техника, которая не включалась в разметку, при этом данные объекты стали полезны, т.к. они похожи на большой автомобиль, но не являются им;
* в ходе разметки было выявлено не очень большое количество фургонов и грузовых автомобилей. Если с определением грузовых автомобилей каких-то сложностей в дальнейшем возникнуть не должно, то фургоны нейросеть может путать со зданиями, в связи с нечеткими границами объектов и сложным выделением характерных признаков;
* автомобили попадающие в кадр частично, на половину и более также размечаются как объект

**Преобразуем полученные данные из файла csv в метку для каждого изображения**

In [2]:
def VIA2YOLO_detect(data, cls_codes, out_dir, imgsize):
    
    annot = pd.read_csv(data, index_col=0)
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        
    for label, row in annot.iterrows():
        name, r_count, coords, cls = label, int(row[2]), json.loads(row[4]), json.loads(row[5])
        cls = str(cls)
        
        # для изображений без объектов создаем пустые файлы разметки
        if r_count == 0:
            with open(os.path.join(out_dir, label.split('.')[0]) + '.txt', 'w') as f:
                f.write('')
        
        if r_count == 1:
            if coords['name'] == 'rect':
                xywh = []
                xywh.append(coords['x']/imgsize[0] + (coords['width']/imgsize[0])/2)
                xywh.append(coords['y']/imgsize[1] + (coords['height']/imgsize[1])/2)
                xywh.append(coords['width']/imgsize[0])
                xywh.append(coords['height']/imgsize[1])

                for key in cls_codes.keys():
                    if key in cls:
                        cls_id = cls_codes[key]

                fin_str = str(cls_id) + ' ' + str(xywh)[1:-1].replace(',','')

                with open(os.path.join(out_dir, label.split('.')[0]) + '.txt', 'w') as f:
                    f.write(fin_str + '\n')
            
        if r_count > 1:
            if coords['name'] == 'rect':
                xywh = []
                xywh.append(coords['x']/imgsize[0] + (coords['width']/imgsize[0])/2)
                xywh.append(coords['y']/imgsize[1] + (coords['height']/imgsize[1])/2)
                xywh.append(coords['width']/imgsize[0])
                xywh.append(coords['height']/imgsize[1])

                for key in cls_codes.keys():
                    if key in cls:
                        cls_id = cls_codes[key]

                fin_str = str(cls_id) + ' ' + str(xywh)[1:-1].replace(',','')

                with open(os.path.join(out_dir, label.split('.')[0]) + '.txt', 'a') as f:
                    f.write(fin_str + '\n')
    
    return annot['region_attributes'].value_counts()

In [3]:
data = 'via_project_16Nov2024_21h42m_csv.csv'
out_dir = 'labels' # директория с метками
# директория с изображениям "images/"

mcodes = {'car': 0} # единственный класс

VIA2YOLO_detect(data, mcodes, out_dir, (640, 640))

C:\Users\al913\AppData\Local\Temp\ipykernel_10088\2728965611.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name, r_count, coords, cls = label, int(row[2]), json.loads(row[4]), json.loads(row[5])


region_attributes
{"Object":"car"}    434
{}                    4
Name: count, dtype: int64

Размеченные изображения перемещены в директорию "images", файлы разметки - в "labels".

### 2. Разделение изображений на обучающую и валидационную выборки

**Разделение изображений производим с помощью скрипта _rnd_split_**

In [4]:
def rnd_split(img_data, lab_data, replace, val_frac):
    if val_frac >= 1:
        val_frac = 1
        print(f'Доля распределения {val_frac}, но должна быть менее 1')
        
    imgs = os.listdir(img_data)
    labs = os.listdir(lab_data)
        
    if not os.path.exists(os.path.join(img_data, 'train')):
        os.mkdir(os.path.join(img_data, 'train'))
    if not os.path.exists(os.path.join(img_data, 'val')):
        os.mkdir(os.path.join(img_data, 'val'))
    if not os.path.exists(os.path.join(lab_data, 'train')):
        os.mkdir(os.path.join(lab_data, 'train'))
    if not os.path.exists(os.path.join(lab_data, 'val')):
        os.mkdir(os.path.join(lab_data, 'val'))
    
    val = int(len(imgs)*val_frac)
    rs = torch.utils.data.RandomSampler
    val_sub = [imgs[i].split('.')[0] for i in rs(imgs, replace, val)]
    
    for file in imgs:
        if file.split('.')[0] in val_sub:
            shutil.move(os.path.join(img_data, file), os.path.join(img_data, 'val', file))
        else:
            shutil.move(os.path.join(img_data, file), os.path.join(img_data, 'train', file))
            
    for file in labs:
        if file.split('.')[0] in val_sub:
            shutil.move(os.path.join(lab_data, file), os.path.join(lab_data, 'val', file))
        else:
            shutil.move(os.path.join(lab_data, file), os.path.join(lab_data, 'train', file))
            
    return val_sub

In [5]:
img = 'images'
lbl = 'labels'

sub_data = rnd_split(img, lbl, False, 0.2)

### 3. Увеличение обучающей выборки с помощью аугментации

#### 3.1 Создание аугментированных данных
**Итоговый размер датасета составил 125 изображений. Для увеличения размера выборки до 200 изображений используем аугментацию с сохранением разметки, применив ее к изображениям обучающей выборки. Для этого используем библиотеку _Albumentations_.**

In [ ]:
# Пути к папкам
images_path = os.path.join("images", "train")
labels_path = os.path.join("labels", "train")
output_path = "aug_data"

# Параметр: во сколько раз увеличить выборку
augmentation_factor = 2  # Укажите нужное количество

# Создаем папку для сохранения результатов, если она не существует
os.makedirs(output_path, exist_ok=True)

# Определяем список аугментаций
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=0.1, 
        scale_limit=0.2, 
        rotate_limit=15, 
        border_mode=0, 
        p=0.5
    ),
    A.GaussNoise(p=0.2)
], bbox_params=A.BboxParams(
    format='yolo',
    label_fields=['class_labels'],
    min_area=0.0,
    min_visibility=0.0,
    check_each_transform=False  # Отключаем проверку после каждого преобразования
))

# Функция для чтения разметки из файла
def read_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            data = line.strip().split()
            class_id = int(data[0])
            x_center, y_center, width, height = map(float, data[1:])
            labels.append((class_id, x_center, y_center, width, height))
    return labels

# Функция для сохранения разметки в файл
def save_labels(label_file, bboxes):
    with open(label_file, 'w') as f:
        for class_id, bbox in bboxes:
            x_center, y_center, w, h = bbox
            f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# Функция для коррекции bounding boxes
def clip_bbox(bbox):
    """
    Обрезка координат bounding box до диапазона [0.0, 1.0].
    Если после обрезки ширина или высота становятся <= 0, возвращаем None.
    """
    x_center, y_center, w, h = bbox

    # Вычисляем x_min, y_min, x_max, y_max
    x_min = x_center - w / 2
    y_min = y_center - h / 2
    x_max = x_center + w / 2
    y_max = y_center + h / 2

    # Обрезаем до диапазона [0.0, 1.0]
    x_min = max(0.0, x_min)
    y_min = max(0.0, y_min)
    x_max = min(1.0, x_max)
    y_max = min(1.0, y_max)

    # Пересчитываем ширину и высоту
    w = x_max - x_min
    h = y_max - y_min

    # Проверяем, что ширина и высота больше 0
    if w <= 0 or h <= 0:
        return None

    # Пересчитываем центр
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2

    return x_center, y_center, w, h

# Функция для аугментации изображения и разметки
def augment_image(image, bboxes):
    # Подготовка bounding boxes для Albumentations
    bbox_list = []
    class_labels = []
    for b in bboxes:
        class_id, x_center, y_center, w, h = b
        clipped_bbox = clip_bbox((x_center, y_center, w, h))
        if clipped_bbox is not None:
            bbox_list.append(clipped_bbox)
            class_labels.append(class_id)
    if not bbox_list:
        # Если нет валидных bounding boxes, возвращаем исходное изображение
        return image, []

    augmented = transform(image=image, bboxes=bbox_list, class_labels=class_labels)

    # Корректируем bounding boxes после аугментации
    corrected_bboxes = []
    for class_id, bbox in zip(augmented['class_labels'], augmented['bboxes']):
        clipped_bbox = clip_bbox(bbox)
        if clipped_bbox is not None:
            corrected_bboxes.append((class_id, clipped_bbox))

    return augmented['image'], corrected_bboxes

# Основной цикл для обработки всех изображений и файлов разметки
for filename in os.listdir(images_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        image_path = os.path.join(images_path, filename)
        label_path = os.path.join(labels_path, filename.rsplit('.', 1)[0] + ".txt")

        # Проверяем наличие файла разметки
        if os.path.exists(label_path):
            # Загружаем изображение
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            # Загружаем разметку
            bboxes = read_labels(label_path)

            # Аугментация несколько раз для увеличения выборки
            for i in range(augmentation_factor):
                # Аугментация изображения и разметки
                aug_image, aug_bboxes = augment_image(image, bboxes)

                if not aug_bboxes:
                    # Если после аугментации нет валидных bounding boxes, пропускаем
                    continue

                # Сохранение аугментированного изображения с уникальным именем
                output_image_path = os.path.join(output_path, f"{filename.rsplit('.', 1)[0]}_aug_{i}.png")
                cv2.imwrite(output_image_path, aug_image)

                # Сохранение аугментированной разметки
                output_label_path = os.path.join(output_path, f"{filename.rsplit('.', 1)[0]}_aug_{i}.txt")
                save_labels(output_label_path, aug_bboxes)

                print(f"Обработано и сохранено: {output_image_path}")

**На выходе получим увеличение количества изображений тренировочной выборки до 180.**
**Полный объем выборки - 204 изображения**

#### 3.2 Проверка фугментированных данных:
**Для проверки просмотрим 20% аугментированных данных**

In [2]:
def convert2cv(data, img_x, img_y):
    params = data.split()
    xc = int(float(params[1]) * img_x)
    yc = int(float(params[2]) * img_y)
    width = int(float(params[3]) * img_x)
    heigth = int(float(params[4]) * img_y)
    return xc, yc, width, heigth

In [3]:
def check_bb(images, labels, name):
    img = cv2.imread(os.path.join(images, name + '.jpg'))
    img_y, img_x = img.shape[0], img.shape[1]
    with open(os.path.join(labels, name + '.txt'), 'r') as l:
        bb = []
        for line in l:
            x, y, w, h = convert2cv(line, img_x, img_y)
            x0, y0 = x - w // 2, y + h // 2
            xw, yh = x + w // 2, y - h // 2
            bb.append(((x0, y0), (xw, yh)))
    for rect in bb:
        cv2.rectangle(img, rect[0], rect[1], (0,255,0), 3)
    cv2.imshow('Bounding boxes', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
pth = os.curdir
images = os.path.join('frames', 'Аугментация', 'images')
labels = os.path.join('frames', 'Аугментация', 'labels')
files = os.listdir(os.path.join(pth, images))
step = int(1 / 0.2) # 20% изображений
for i in range(0, len(files), step):
    check_bb(images, labels, files[i].split('.')[0])

**Разметка аугментированной выборки соответсвует необходимым условиям**

### 4. Создание структуры проекта

#### 4.1 Сруктура проекта 

**Для проекта используется следующая структура**

#### 4.2 Содержимое файле **data.yaml** - файл путей и переменных:
* в переменной ```path``` абсолютный путь к корневой директории Проекта **data**
* в соответствующих переменных - относитльные адреса директорий с изображениями и разметкой в соответствии с выбранной структурой.
*  в переменной ```names``` - индексы и номера классов в формате ```0: name_0```, ```1: name_1``` ... и т.д., где name_0, name_1... - реальные имена ваших классов. В данной задаче объекты представлены единственным классом **```car```**.

#### 4.3 Содержимое файле **default.yaml** - файл настроек:
** Основные параметры**:
* Задача обучения - детекция
* Модель - предобученная, yolo8n
* Обучение на 20 эпохах
* Время эпох переопределения эпох
* Размер батча - 16
* Размер изображения 640х640
* Заданы гиперпараметры:
  * lr0=lrf=0,01 (lerning rate не меняется в процессе обучения);
  * momentum = 0,937;
  * оптимизатор весов = 0.0005 и др.

### 5. Запуск обучения

**Фикисруем источники случайности**

In [2]:
seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
os.environ['NUMEXPR_NUM_THREADS'] = '8'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

**Версия pytorch, доступность видеокарты**

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch version: {torch.__version__}, device: {device}')

torch version: 2.5.1+cpu, device: cpu


**Проверка настроек модели**

In [7]:
settings
# settings['datasets_dir']
# settings['runs_dir']

{'settings_version': '0.0.6',
 'datasets_dir': 'C:\\Users\\al913\\Downloads\\БАС\\venv\\БАС\\00_ДЗ\\ДЗ_9_Практика_Свод\\03_Аттестация\\datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '9eed85bafc3f1a44f2784cd498a4782724d5e79b20fb1c9ce361c9e00a1f45e1',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': False,
 'vscode_msg': True}

**Инициализация модели YOLO8 nano (предобученной)**

In [4]:
model_npt = YOLO("yolov8n.pt")

**Визуализация структуры модели**

In [5]:
model_npt

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

**Обучение модели**

In [10]:
model_npt.train(data='data.yaml', cfg='default.yaml')
# model_npt.train(data='data.yaml', batch=16, epochs=1, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.34 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.33  Python-3.12.7 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=attestation-5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=True, save_fr

train: Scanning C:\Users\al913\Downloads\БАС\БАС\00_ДЗ\ДЗ_9_Практика_Свод\03_Аттестация\model\labels\train.cache... 180

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\al913\Downloads\БАС\БАС\00_ДЗ\ДЗ_9_Практика_Свод\03_Аттестация\model\labels\val.cache... 24 imag


Plotting labels to runs\detect\attestation-5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\attestation-5
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         24        150      0.898      0.793      0.886      0.618



30 epochs completed in 0.498 hours.
Optimizer stripped from runs\detect\attestation-5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\attestation-5\weights\best.pt, 6.2MB

Validating runs\detect\attestation-5\weights\best.pt...
Ultralytics 8.3.33  Python-3.12.7 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<0


                   all         24        150      0.902      0.798      0.898      0.628
Speed: 1.3ms preprocess, 73.8ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs\detect\attestation-5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000191C47548C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

### 6. Валидация модели

**Проверка модели**

In [11]:
metrics = model_npt.val()
metrics

Ultralytics 8.3.33  Python-3.12.7 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\al913\Downloads\БАС\БАС\00_ДЗ\ДЗ_9_Практика_Свод\03_Аттестация\model\labels\val.cache... 24 imag
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<0


                   all         24        150      0.902      0.798      0.898      0.628
Speed: 4.3ms preprocess, 80.0ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs\detect\attestation-52


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000191C9F28B30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

### 7. Инференс модели

**Запуск проверки модели на новых данных (изображения)**

In [12]:
folder = 'inference'
results = model_npt.predict(source=folder, save=True, save_txt=True)


Saving runs\detect\attestation-53\for_add_img_0\stage0_Conv_features.png... (16/16)
Saving runs\detect\attestation-53\for_add_img_0\stage1_Conv_features.png... (32/32)
Saving runs\detect\attestation-53\for_add_img_0\stage2_C2f_features.png... (32/32)
Saving runs\detect\attestation-53\for_add_img_0\stage3_Conv_features.png... (32/64)
Saving runs\detect\attestation-53\for_add_img_0\stage4_C2f_features.png... (32/64)
Saving runs\detect\attestation-53\for_add_img_0\stage5_Conv_features.png... (32/128)
Saving runs\detect\attestation-53\for_add_img_0\stage6_C2f_features.png... (32/128)
Saving runs\detect\attestation-53\for_add_img_0\stage7_Conv_features.png... (32/256)
Saving runs\detect\attestation-53\for_add_img_0\stage8_C2f_features.png... (32/256)
Saving runs\detect\attestation-53\for_add_img_0\stage9_SPPF_features.png... (32/256)
Saving runs\detect\attestation-53\for_add_img_0\stage10_Upsample_features.png... (32/256)
Saving runs\detect\attestation-53\for_add_img_0\stage11_Concat_featu

**Запуск проверки модели на новых данных (видео)**

In [ ]:
vid = 'for_add_img.mp4'
results = model_npt.predict(source=vid, show=True)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/11605) C:\Users\al913\Downloads\\\00_\_9__\03_\model\for_add_img.mp4: 384x640 (no detections), 86.0ms
video 1/1 (frame 2/11605) C:\Users\al913\Downloads\\\00_\_9__\03_\model\for_add_img.mp4: 384x640 (no detections), 61.5ms
video 1/1 (frame 3/11605) C:\Users\al913\Downloads\\\00_\_9__\03_\model\for_add_img.mp4: 384x640 (no detections), 60.9ms
video 1/1 (frame 4/11605) C:\Users\al913\Downloads\\\00_\_9__\03_\model\for_add_img.mp4: 384x64